In [1]:
COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  local_path = '/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets'
else:
  local_path = r'G:\Mi unidad\ProyectoIA2\generative-spectral-datasets'

import os 
os.chdir(local_path)

Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np

from models.main import make_autoencoder, make_discriminator
from models.metrics import PSNR, SSIM
from data import load_dataset

input_shape = (512,512,31)

In [4]:
BATCH_SIZE = 3

train_params = dict(
    batch_size = BATCH_SIZE,
    shuffle = True,
    cache = True
)

test_params = dict(
    batch_size = BATCH_SIZE,
    cache = True
)

train_ds , test_ds = load_dataset( train_params = train_params, test_params = test_params)

In [5]:
FACTOR = [ 1 , 1 , 1/2 , 1/2 , 1/4, 1/8]

features= 64

autoencoder = make_autoencoder(input_shape, features=features, factors=FACTOR)
mse_loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(1e-4)

metrics = [PSNR(), SSIM()]
autoencoder.compile( optimizer=optimizer, loss=mse_loss, metrics=metrics)

In [6]:
autoencoder.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 31)]    0         
_________________________________________________________________
encoder (Encoder)            (None, 512, 512, 8)       59184     
_________________________________________________________________
generator (Generator)        (None, 512, 512, 31)      59230     
Total params: 118,414
Trainable params: 118,414
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.fit( train_ds , epochs=300, validation_data=test_ds)

Epoch 1/300
9/9 [==============================] - 144s 3s/step - loss: 0.0115 - psnr: 20.1024 - ssim: 0.5178 - val_loss: 0.0108 - val_psnr: 20.0839 - val_ssim: 0.5808
Epoch 2/300
9/9 [==============================] - 11s 1s/step - loss: 0.0092 - psnr: 21.1849 - ssim: 0.5594 - val_loss: 0.0095 - val_psnr: 20.6566 - val_ssim: 0.5830
Epoch 3/300
9/9 [==============================] - 11s 1s/step - loss: 0.0082 - psnr: 21.6223 - ssim: 0.5766 - val_loss: 0.0089 - val_psnr: 20.9307 - val_ssim: 0.6020
Epoch 4/300
9/9 [==============================] - 11s 1s/step - loss: 0.0077 - psnr: 21.9417 - ssim: 0.6080 - val_loss: 0.0083 - val_psnr: 21.2509 - val_ssim: 0.6351
Epoch 5/300
9/9 [==============================] - 11s 1s/step - loss: 0.0071 - psnr: 22.2940 - ssim: 0.6335 - val_loss: 0.0077 - val_psnr: 21.5638 - val_ssim: 0.6470
Epoch 6/300
9/9 [==============================] - 11s 1s/step - loss: 0.0066 - psnr: 22.6311 - ssim: 0.6428 - val_loss: 0.0072 - val_psnr: 21.8673 - val_ssim: 0.65